# Convert TEI into TSV


In [14]:
import re
import os
from lxml import etree
import pandas as pd

In [15]:
def clean_text(text):
    # Remove leading and trailing whitespace
    text = text.strip()
    # Replace newlines with a space
    text = text.replace("\n", " ")
    # Split the text into words and join them with a single space
    text = " ".join(text.split())
    return text

def extract_text(element):
    text = element.text or ""
    for child in element:
        text += extract_text(child)
        if child.tail:
            text += child.tail
    return clean_text(text)



In [16]:
inputpath = os.path.join('..', 'data', '1743_LeRobert', 'tei')

data = []

for filename in sorted(os.listdir(inputpath)):
    try:
        parser = etree.XMLParser(collect_ids=False, encoding='utf-8') 
        root = etree.parse(os.path.join(inputpath, filename), parser=parser).getroot()    
        #print(root.nsmap)
        print(filename)
        volume = filename[2:3]

        for entry in root.findall('.//entry[@type="mainEntry"]', namespaces=root.nsmap):
            usg = None
            id = 1
            entry_id = entry.get('id')
            form = entry.find('.//form[@type="lemma"]/orth', namespaces=root.nsmap)
            if form is not None:
                entry_lemma = form.text
            else:
                print("Forme : non trouvée, entry:", entry_id)
                entry_lemma = None
            entry_content = ""
            for i, subordinate in enumerate(entry.findall('./sense', namespaces=root.nsmap)):
                
                form = subordinate.find('.//form/orth', namespaces=root.nsmap)
                if form is not None:
                    subordinate_lemma = form.text
                else:
                    subordinate_lemma = None
                if usg is None:
                    usg = subordinate.find('.//usg[@type="domain"]', namespaces=root.nsmap)
                    if usg is not None:
                        subordinate_domain = usg.text
                    else:
                        subordinate_domain = None

                # Extract the text content from the subordinate element
                content = extract_text(subordinate)

                if i == 0:
                    content = entry_lemma + " " + content

                entry_content += content + "\n\n"

            if entry_content != "":
                row = [volume, entry_id, entry_lemma, id, 'mainEntry', subordinate_lemma, subordinate_domain, entry_content]
                #volume | entry | entry_lemma | subordinate | subordinate_lemma | subordinate_domain | content
                data.append(row)
                

            for relatedEntry in entry.findall('.//entry[@type="relatedEntry"]', namespaces=root.nsmap):
                id += 1
                usg = None
                relatedEntry_content = ""
                form = relatedEntry.find('.//form/orth', namespaces=root.nsmap)
                if form is not None:
                    subordinate_lemma = form.text
                else:
                    subordinate_lemma = None
                if usg is None:
                    usg = subordinate.find('.//usg[@type="domain"]', namespaces=root.nsmap)
                    if usg is not None:
                        subordinate_domain = usg.text
                    else:
                        subordinate_domain = None

                for i, subordinate in enumerate(relatedEntry.findall('.//sense', namespaces=root.nsmap)):
                    
                    form = subordinate.find('.//form/orth', namespaces=root.nsmap)
                    if form is not None:
                        subordinate_lemma = form.text
                    else:
                        subordinate_lemma = None

                    usg = subordinate.find('.//usg[@type="domain"]', namespaces=root.nsmap)
                    if usg is not None:
                        subordinate_domain = usg.text
                    else:
                        subordinate_domain = None

                    # Extract the text content from the subordinate element
                    content = extract_text(subordinate)

                    if i == 0:
                        content = entry_lemma + " " + content

                    relatedEntry_content += content + "\n\n"

                row = [volume, entry_id, entry_lemma, id, 'relatedEntry', subordinate_lemma, subordinate_domain, relatedEntry_content]
                #volume | entry | entry_lemma | subordinate | subordinate_lemma | subordinate_domain | content
                data.append(row)
                    
                #print(row)
        
    except etree.XMLSyntaxError as e:
        print(f"Erreur de syntaxe XML : {e}")


# convert data into a dataframe

df = pd.DataFrame(data, columns=['volume', 'entry', 'entry_lemma', 'id', 'type', 'subordinate_lemma', 'subordinate_domain', 'content'])
df.head()


Erreur de syntaxe XML : Document is empty, line 1, column 1 (.DS_Store, line 1)
TR1.tei
TR2.tei
TR3.tei
TR4.tei
TR5.tei
TR6.tei


,volume,entry,entry_lemma,id,type,subordinate_lemma,subordinate_domain,content
0,1,250000010,A,1,mainEntry,None,None,A est la première Lettre de l'Alphabet Françoi...
1,1,250000020,AAHUS,1,mainEntry,None,None,AAHUS Aahusium. Ville de l'Evéché de Munster. ...
2,1,250000030,AAR,1,mainEntry,None,None,"AAR ou AHR. subst. Aara, Abrinca. Rivière d'Al..."
3,1,250000040,AAR,1,mainEntry,None,None,"AAR Arula ou Arola, & non pas Arosa, comme on ..."
4,1,250000050,AARBRER,1,mainEntry,None,None,AARBRER Terme ancien qui n'est pas aujourd'hui...


In [17]:
df.shape

(81374, 8)

In [18]:
df.head(60)

,volume,entry,entry_lemma,id,type,subordinate_lemma,subordinate_domain,content
0,1,250000010,A,1,mainEntry,None,None,A est la première Lettre de l'Alphabet Françoi...
1,1,250000020,AAHUS,1,mainEntry,None,None,AAHUS Aahusium. Ville de l'Evéché de Munster. ...
2,1,250000030,AAR,1,mainEntry,None,None,"AAR ou AHR. subst. Aara, Abrinca. Rivière d'Al..."
3,1,250000040,AAR,1,mainEntry,None,None,"AAR Arula ou Arola, & non pas Arosa, comme on ..."
4,1,250000050,AARBRER,1,mainEntry,None,None,AARBRER Terme ancien qui n'est pas aujourd'hui...
5,1,250000060,AARON,1,mainEntry,None,None,AARON Aaron. Nom propre qu'il faut prononcer c...
6,1,250000070,AB,1,mainEntry,None,None,"AB Cinquième mois des Hébreux, qui répond à no..."
7,1,250000070,AB,2,relatedEntry,Ab,None,"AB Ab , en Langue Syriaque, le dernier mois de..."
8,1,250000080,ABA,1,mainEntry,None,None,"ABA Aba, ou Abae. C'est le nom d'une ville de ..."
9,1,250000090,ABA,1,mainEntry,None,None,"ABA ou Anba, Pere, titre que les Églises Syria..."


In [19]:
df['book'] = 'DUFLT_1743'

df.rename(columns={"entry_lemma": "head", "content": "text", "subordinate_domain":"src-domain"}, inplace=True)
df['numero'] = df.groupby('volume')['entry'].transform(lambda x: pd.factorize(x)[0] + 1)

df = df[['book', 'volume', 'numero', 'head', 'id', 'type', 'src-domain', 'text']]


In [20]:
df.head(60)

,book,volume,numero,head,id,type,src-domain,text
0,DUFLT_1743,1,1,A,1,mainEntry,None,A est la première Lettre de l'Alphabet Françoi...
1,DUFLT_1743,1,2,AAHUS,1,mainEntry,None,AAHUS Aahusium. Ville de l'Evéché de Munster. ...
2,DUFLT_1743,1,3,AAR,1,mainEntry,None,"AAR ou AHR. subst. Aara, Abrinca. Rivière d'Al..."
3,DUFLT_1743,1,4,AAR,1,mainEntry,None,"AAR Arula ou Arola, & non pas Arosa, comme on ..."
4,DUFLT_1743,1,5,AARBRER,1,mainEntry,None,AARBRER Terme ancien qui n'est pas aujourd'hui...
5,DUFLT_1743,1,6,AARON,1,mainEntry,None,AARON Aaron. Nom propre qu'il faut prononcer c...
6,DUFLT_1743,1,7,AB,1,mainEntry,None,"AB Cinquième mois des Hébreux, qui répond à no..."
7,DUFLT_1743,1,7,AB,2,relatedEntry,None,"AB Ab , en Langue Syriaque, le dernier mois de..."
8,DUFLT_1743,1,8,ABA,1,mainEntry,None,"ABA Aba, ou Abae. C'est le nom d'une ville de ..."
9,DUFLT_1743,1,9,ABA,1,mainEntry,None,"ABA ou Anba, Pere, titre que les Églises Syria..."


In [21]:
df.to_csv(os.path.join('..', 'data', '1743_LeRobert', 'Trevoux1743.tsv'), sep='\t', index=False, encoding='utf-8')


In [22]:
df.shape

(81374, 8)

In [23]:
df.iloc[0,7]

"A est la première Lettre de l'Alphabet François. Elle répond entièrement à la Lettre A des Latins, qui est, comme Jules Scaliger l'a remarqué, le premier son articulé que la nature pousse, le premier cri des Enfans, & qui n'a point besoin d'autre mouvement que de celui d'ouvrir la bouche.\n\nC'est inutilement que la plupart des Grammairiens comparent la Lettre a des Latins & des François, avec l'aleph des Hébreux & l'eliph des Arabes ; parceque ces deux Lettres n'ont aucun rapport avec notre a, si ce n'est qu'elles sont les premières dans l'Alphabet Hébreu & dans celui des Arabes ; mais elles ne sont pas des voyelles comme dans la Langue Françoise. Quelques Auteurs disent que la lettre aleph n'est ni une consonne, ni une voyelle ; mais qu'elle répond à ce que les Grammairiens appellent esprit. Et c'est ce qui a fait dire à Vossius que chez les Hébreux l'aleph, le he, le heth. le hain, ne sont point proprement des Lettres, mais des esprits ; ensorte que l'aleph est ce qu'on nomme commu